<a href="https://colab.research.google.com/github/pranav-1-6/ASR_Wav2Vec2_XLSR_Implementaion_Hindi/blob/main/ASR_Wav2Vec2_XLSR_Implementaion_Hindi_checkpoint_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install datasets --upgrade
!pip install transformers==4.4.0
!pip install transformers --upgrade
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install transformers[torch]
!pip install accelerate

In [ ]:
!pip install git+https://github.com/huggingface/accelerate
!pip install --upgrade transformers

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-u0stgb0a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-u0stgb0a
  Resolved https://github.com/huggingface/accelerate to commit ba5f49219f0681e7a7feeaca565af0e8315a047d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.30.0.dev0-py3-none-any.whl size=301852 sha256=180ea6fca993fb9bbf09675f324e3c3508d2555914a4e93696ccbbd0d77eee59
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzcq105y/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


Prepare Data, Tokenizer, Feature Extractor

In [ ]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4361it [00:00, 83130.45it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2179it [00:00, 89659.86it/s]


...


Reading metadata...: 2894it [00:00, 78088.53it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3328it [00:00, 78743.61it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 680it [00:00, 79822.19it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,"कपिल ने की ऐसी हरकत, गुत्थी शो छोड़ने पर हुए मजबूर!"
1,क्या आप हिंदी में बात कर सकते हैं
2,संबंधों के बीच खूब खनकता है रूप का सिक्का
3,"हिट एंड रन: जल्द आएगा सलमान पर फैसला, हाई कोर्ट ने लिखवाना किया शुरू"
4,सोचो कि तुम्हारे पास एक टाइम मशीन है।
5,बहाल हुई पुलवामा हमले के बाद बंद हुई श्रीनगर-मुजफ्फराबाद बस सेवा
6,रतन टाटा से सीखें जिंदगी में सफलता के गुर...
7,हमने कल रात एक भयानक फ़िल्म देखी।
8,पेंट अभी तक सूखा नहीं है।
9,चीन की जनसंख्या भारत की से बड़ी है।


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/6540 [00:00<?, ? examples/s]

Map:   0%|          | 0/2894 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,कर्नाटक सरकार ने लागू किया ड्रेस कोड 'महिलाएं पहनें साड़ी और चूड़ीदार कुर्ता
1,खुले पत्र के लिए तृणमूल ने टाटा को चेताया
2,वह पैसेवाला तो है पर भरोसेमंद नहीं है।
3,टॉम हमेशा काले कपड़े पहनता है।
4,केरल में धूमधाम से मनाया गया हिंदू नववर्ष 'विषु'
5,कैंसर पीड़ित सोनाली को देखने सबसे पहले न्यूयॉर्क पहुंचा ये स्टार
6,प्री मानसून बारिश के चलते भीगी किसानों की करोड़ों की फसल
7,अरे आप कैसे हैं
8,जब इंदिरा को कहा गया था 'गूंगी गुड़िया' बदल दिया था पाकिस्तान का भूगोल
9,whatsapp अकाउंट एक से ज्यादा स्मार्टफोन्स में यूज कर सकेंगे रिपोर्ट


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/6540 [00:00<?, ? examples/s]

Map:   0%|          | 0/2894 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'झ': 0,
 'त': 1,
 'f': 2,
 'न': 3,
 'ृ': 4,
 'r': 5,
 'ढ': 6,
 'औ': 7,
 's': 8,
 'x': 9,
 'ड': 10,
 'ै': 11,
 'क़': 12,
 'ौ': 13,
 'स': 14,
 'ड़': 15,
 'फ़': 16,
 'ई': 17,
 'म': 18,
 'उ': 19,
 'ग': 20,
 'ग़': 21,
 'ू': 22,
 'l': 23,
 'z': 24,
 'ः': 25,
 'ध': 26,
 'ञ': 27,
 'ऊ': 28,
 '&': 29,
 'भ': 30,
 'क': 31,
 'j': 32,
 'ँ': 33,
 'ख़': 34,
 'ऑ': 35,
 'd': 36,
 'ं': 37,
 'v': 38,
 'च': 39,
 'प': 40,
 'ष': 41,
 'g': 42,
 'अ': 43,
 'ऐ': 44,
 'द': 45,
 'ठ': 46,
 'a': 47,
 '|': 48,
 'व': 49,
 'k': 50,
 'ा': 51,
 'श': 52,
 'े': 53,
 'ज': 54,
 'n': 55,
 'i': 56,
 'e': 57,
 'q': 58,
 'u': 59,
 'ण': 60,
 'ह': 61,
 'ॅ': 62,
 ' ': 63,
 'ो': 64,
 'y': 65,
 'थ': 66,
 '़': 67,
 'h': 68,
 'ऋ': 69,
 'ॉ': 70,
 'ढ़': 71,
 '्': 72,
 'ख': 73,
 '–': 74,
 'o': 75,
 'फ': 76,
 'ी': 77,
 'b': 78,
 'आ': 79,
 '।': 80,
 'घ': 81,
 'ु': 82,
 'ओ': 83,
 'छ': 84,
 'ब': 85,
 't': 86,
 'w': 87,
 'p': 88,
 'ए': 89,
 'ज़': 90,
 'ि': 91,
 'य': 92,
 'इ': 93,
 'ट': 94,
 'र': 95,
 'c': 96,
 '’': 97,
 "'": 98,
 'ल': 99,
 'm': 100}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

102

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

Create Wav2Vec2CTCTokenizer

In [ ]:

from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Create XLSR-Wav2Vec2 Feature Extractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Preprocess Data

In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/c090ee7e53af621f387a7ad442373c879defc9f799c91a68edc0f646c36f115f/hi_train_0/common_voice_hi_26008353.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c090ee7e53af621f387a7ad442373c879defc9f799c91a68edc0f646c36f115f/hi_train_0/common_voice_hi_26008353.mp3',
 'array': array([ 5.81611368e-26, -1.48634016e-25, -9.37040538e-26, ...,
         1.06425901e-07,  4.46416450e-08,  2.61450239e-09]),
 'sampling_rate': 48000}

In [ ]:
common_voice_train[0]["sentence"]

'हमने उसका जन्मदिन मनाया। '

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/c090ee7e53af621f387a7ad442373c879defc9f799c91a68edc0f646c36f115f/hi_train_0/common_voice_hi_26008353.mp3',
 'array': array([ 3.81639165e-17,  2.42861287e-17, -1.73472348e-17, ...,
        -1.30981789e-07,  2.63096808e-07,  4.77157300e-08]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)
print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


जानिये कौन हैं जितेंद्र तोमर और क्या है वो विवाद जिसमें मंत्रीजी हुए गिरफ्तार 


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: लोहा सोने से ज़्यादा उपयोगी होता है। 
Input array shape: (66816,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
target_test = common_voice_test["sentence"]
print(target_test[0])
with processor.as_target_processor():
  target_text = processor(common_voice_test["sentence"]).input_ids
print(target_text[0])

वाराणसी हिंसा पुलिस की भूमिका संदिग्ध वायरल हुई तस्वीरें 


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


[49, 51, 95, 51, 60, 14, 77, 63, 61, 91, 37, 14, 51, 63, 40, 82, 99, 91, 14, 63, 31, 77, 63, 30, 22, 18, 91, 31, 51, 63, 14, 37, 45, 91, 20, 72, 26, 63, 49, 51, 92, 95, 99, 63, 61, 82, 17, 63, 1, 14, 72, 49, 77, 95, 53, 37, 63]


In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/6540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2894 [00:00<?, ? examples/s]

Training

In [ ]:
#setup Trainer

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

WER Metric

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-32-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# import nltk

# def compute_metrics(pred):
#     pred_logits = pred.predictions
#     pred_ids = np.argmax(pred_logits, axis=-1)

#     pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

#     pred_str = processor.batch_decode(pred_ids)
#     label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

#     wer = wer_metric.compute(predictions=pred_str, references=label_str)

#     # Compute BLEU score
#     bleu_score = compute_bleu_score(pred_str, label_str)

#     return {"wer": wer, "bleu": bleu_score}

# def compute_bleu_score(pred_str, label_str):
#     """Compute the BLEU score between predicted and reference sentences.

#     Args:
#         pred_str (List[str]): List of predicted sentences.
#         label_str (List[str]): List of reference sentences.

#     Returns:
#         float: BLEU score.
#     """
#     # Tokenize predicted and reference sentences
#     pred_tokens = [nltk.word_tokenize(pred.lower()) for pred in pred_str]
#     label_tokens = [nltk.word_tokenize(label.lower()) for label in label_str]

#     # Compute BLEU score
#     bleu_score = nltk.translate.bleu_score.corpus_bleu([label_tokens], pred_tokens)

#     return bleu_score

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(

  output_dir="./wav2vec2-xlsr-Hindi",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  # fp16_full_eval=False,  # Disable mixed precision evaluation
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,7.755700,6.711102,1.000000
200,3.547500,3.548626,1.000000
300,3.460200,3.460082,1.000000
400,3.419000,3.400758,1.000000
500,2.174400,2.017924,1.039113
600,1.030900,1.190069,0.909414
700,0.743200,0.875524,0.745090


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call